Deep Neural Network 
=================================================
< Training with the MNIST Data, and Testing with My Own Data >
----------------------


In [1]:
import os

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import cv2

### MNIST Datasets

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### Setting the Number of Nodes and the Batch size

In [3]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
n_classes = 10

batch_size = 100

### Setting the Placeholder and Variables

In [4]:
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

w1 = tf.Variable(tf.random_normal([784, n_nodes_hl1]))
b1 = tf.Variable(tf.random_normal([n_nodes_hl1]))

w2 = tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2]))
b2 = tf.Variable(tf.random_normal([n_nodes_hl2]))

w3 = tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3]))
b3 = tf.Variable(tf.random_normal([n_nodes_hl3]))

w4 = tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]))
b4 = tf.Variable(tf.random_normal([n_classes]))

### A Network Model

In [5]:
l1 = tf.add(tf.matmul(x,w1), b1)
l1 = tf.nn.relu(l1)

l2 = tf.add(tf.matmul(l1,w2), b2)
l2 = tf.nn.relu(l2)

l3 = tf.add(tf.matmul(l2,w3), b3)
l3 = tf.nn.relu(l3)

output = tf.matmul(l3,w4) + b4

### Set the Path of a Metafile

In [6]:
save_path = 'machine/'
model_name = 'DNN_md'
if not os.path.exists(save_path):
    os.makedirs(save_path)

save_path_full = os.path.join(save_path, model_name)

### Training

In [7]:
def train_neural_network(x):
    prediction = output
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    learning_rate=0.001
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    hm_epochs = 25
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                sess.run(l1, feed_dict={x: epoch_x})
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

        saver = tf.train.Saver([w1, w2, w3, w4, b1, b2, b3, b4])
        saver.save(sess,save_path_full)
        
train_neural_network(x)

Epoch 0 completed out of 25 loss: 1628960.76965
Epoch 1 completed out of 25 loss: 410505.061989
Epoch 2 completed out of 25 loss: 230485.836246
Epoch 3 completed out of 25 loss: 128404.928162
Epoch 4 completed out of 25 loss: 79694.5551399
Epoch 5 completed out of 25 loss: 52964.1649789
Epoch 6 completed out of 25 loss: 33736.2722233
Epoch 7 completed out of 25 loss: 25091.673692
Epoch 8 completed out of 25 loss: 23077.8821133
Epoch 9 completed out of 25 loss: 17107.081597
Epoch 10 completed out of 25 loss: 17994.9481698
Epoch 11 completed out of 25 loss: 17416.3082521
Epoch 12 completed out of 25 loss: 15907.0713582
Epoch 13 completed out of 25 loss: 15763.4264113
Epoch 14 completed out of 25 loss: 13330.8049584
Epoch 15 completed out of 25 loss: 11726.7031744
Epoch 16 completed out of 25 loss: 11327.3681108
Epoch 17 completed out of 25 loss: 11055.1103503
Epoch 18 completed out of 25 loss: 12740.6378195
Epoch 19 completed out of 25 loss: 11669.947741
Epoch 20 completed out of 25 loss

### Test Datasets

#### Original Data
<p>(184 × 326)</p>
<p><img src="https://raw.githubusercontent.com/symoon94/Image_Data/master/image_7.png" align="left" "vertical-align:middle" </p>


In [8]:
# load an image as gray scale and resize the image
img = cv2.imread("/Users/moonsooyoung/Desktop/image_7.png",0)    # 0 = gray

rsz_img = cv2.resize(img, (28,28), interpolation=cv2.INTER_AREA)
cv2.imshow("resize", rsz_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

<img src="https://raw.githubusercontent.com/symoon94/Image_Data/master/image_7_rsz.png" align="left">


In [9]:
image = cv2.adaptiveThreshold(rsz_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY_INV,11,2)
cv2.imshow('inv',image)
cv2.waitKey(0)
cv2.destroyAllWindows()


<img src="https://raw.githubusercontent.com/symoon94/Image_Data/master/image_7_inv.png" align="left">

In [10]:
data=image.reshape(1,784)
data = data/255    # 255 -> 1
x_data = tf.cast(data, 'float')

In [11]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.import_meta_graph('machine/DNN_md.meta')
saver.restore(sess,save_path_full)

INFO:tensorflow:Restoring parameters from machine/DNN_md


In [12]:
l1 = tf.add(tf.matmul(x_data,w1), b1)    # x: x_data
l1 = tf.nn.relu(l1)

l2 = tf.add(tf.matmul(l1,w2), b2)
l2 = tf.nn.relu(l2)

l3 = tf.add(tf.matmul(l2,w3), b3)
l3 = tf.nn.relu(l3)

output = tf.matmul(l3,w4) + b4

### Results

In [13]:
print(sess.run (output))
print(sess.run(tf.argmax(output, 1)))

[[-45109.69921875 -34030.3203125  -17227.49609375   6150.22412109
    7038.12402344 -25238.21679688 -39544.28125     31142.71484375
  -13743.43066406    311.59197998]]
[7]
